# ML cube Platform SDK - Production Routine

In this notebook, you will see how to setup a production routine, a production routine is composed of sending production data, check the model status and then ask the retraining report if it has drifted.

**Requirements**:

1. API Key with roles `COMPANY_ADMIN` or `PROJECT_ADMIN`
2. Id of the task
3. Id and version of the model

**User Input**

In the notebook you will need to complete variables and names to correctly run it.
Whenever you see the comment `# TO COMPLETE` you need to fill the empty string.

**Define entities ids**

Specify here the ids of the entities that are required to work on this notebook.

In [ ]:
# TO COMPLETE
task_id = ""
model_id = ""
model_version = ""

**Imports**

In [ ]:
import logging
logger = logging.getLogger("platform_tutorial")

In [ ]:
from ml3_platform_sdk.client import ML3PlatformClient
from ml3_platform_sdk import enums as ml3_enums
from ml3_platform_sdk import models as ml3_models

**Instantiace the Client**

To interact with ML cube Platform you need to instantiate the client only the first time.
Then you will use its methods to perform requests.
Please, insert the api key we provided you to instantiate the client.

In [ ]:
API_KEY = ""  # TO COMPLETE
URL = "https://api.platform.mlcube.com"
ml3_client = ML3PlatformClient(api_key=API_KEY, url=URL)

**Create Detection Automation Rule**

Production data are analysed by our monitoring algorithms to detect drifts.
You can setup automation rules to receives notifications in your desired communication channel and to retrain automatically your model.

In [ ]:
# TO COMPLETE
rule_id = ml3_client.create_detection_event_rule(
    name='rule1',
    task_id=task_id,
    model_id=model_id,
    severity=ml3_enums.DetectionEventSeverity.MEDIUM,
    detection_event_type=ml3_enums.DetectionEventType.DRIFT,
    monitoring_target=ml3_enums.MonitoringTarget.MODEL,
    actions=[
        ml3_models.DiscordNotificationAction(webhook='discord_webhook'),
        ml3_models.SlackNotificationAction(webhook='slack_webhook', channel='slakc_channel'),
        ml3_models.RetrainAction()
    ],
)
logger.info(f"Created event rule with id {rule_id}")

**Add production data**

Production data are uploaded to ML cube Platform like historical and reference. 
However, in this scenario it is common to have data at different times, for instance, your model receives input data and provides predictions but actual targets will available with a bit of delay.
Then, you have the possibility to upload data as soon as you receive them and ML cube Platform will start monitoring algorithms when data are available.

In [ ]:
# TO COMPLETE
job_id = ml3_client.add_production_data(
    task_id=task_id,
    inputs=ml3_models.TabularData(
        source=ml3_models.LocalDataSource(
            data_structure=ml3_enums.DataStructure.TABULAR,
            file_type=ml3_enums.FileType.CSV,
            file_path="path/to/file.csv",
            is_folder=False,
            folder_type=None,
        )
    ),
    target=None
    predictions=[
        (
            model_id,
            ml3_models.TabularData(
                source=ml3_models.LocalDataSource(
                    data_structure=ml3_enums.DataStructure.TABULAR,
                    file_type=ml3_enums.FileType.CSV,
                    file_path="path/to/file.csv",
                    is_folder=False,
                    folder_type=None,
                )
            ),
        )
    ],
)
logger.info(f'Add production input and predictions data job {job_id}')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Add production input and predictions data job {job_id} terminated')

In [ ]:
job_id = ml3_client.add_production_data(
    task_id=task_id,
    inputs=None,
    target=ml3_models.TabularData(
        source=ml3_models.LocalDataSource(
            data_structure=ml3_enums.DataStructure.TABULAR,
            file_type=ml3_enums.FileType.CSV,
            file_path="path/to/file.csv",
            is_folder=False,
            folder_type=None,
        )
    ),
    predictions=[],
)
logger.info(f'Add production target data job {job_id} started')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Add production target data job {job_id} terminated')

**Check model status and ask retraining report**

When drift is detected, the model status changes from `OK` to `DRIFT`. 
This is the moment we you really need ML cube Platform!
The best action to do is to ask the retraining report that contains all the information to handle the dirft and retrain your model.

In [ ]:
logger.info(f'Read model by id')
model = ml3_client.get_model(model_id=model_id)
logger.info(f'Model: {model}')

In [ ]:
logger.info(f'Compute retraining report')
job_id = ml3_client.compute_retraining_report(model_id=model_id)
logger.info(f'Job created, id {job_id}')

ml3_client.show_jobs()

logger.info(f'Waiting job completion')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Job completed')

logger.info(f'Get retraining report')
retraining_report = ml3_client.get_retraining_report(model_id=model_id)

logger.info(f'Retraining Report: {retraining_report}')

**Retrain your model and update it to ML cube Platform**

When you receive the retraining report, your task is to retrain the model with our importance weights and then you need to notify this to ML cube Platform.

In [ ]:
logger.info("API - Show suggestions")
ml3_client.show_suggestions(model_id=model_id, model_version=model_version)

logger.info(f'API - Update model version from suggestion id')
new_model_version = 'v0.0.2'
job_id = ml3_client.update_model_version_by_suggestion_id(
    model_id=model_id,
    new_model_version=new_model_version,
    suggestion_id=retraining_report.suggestion_id,
)

logger.info(f'Job created, id {job_id}')

logger.info(f'Waiting job completion')
ml3_client.wait_job_completion(job_id=job_id)
logger.info(f'Job completed')

**Congratulations!**

In this notebook, you learned how to create a task, add a model to this task and uploading to ML cube Platform both historical and reference data.